In [0]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/input/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
data = pd.read_csv("../input/google-quest-challenge/train.csv")
test = pd.read_csv("../input/google-quest-challenge/test.csv")


In [0]:
drop_data_temp=data.drop(['question_user_name', 'question_user_page','answer_user_name','answer_user_page','url','host'], axis=1)
drop_data_temp.head()

,qa_id,question_title,question_body,answer,category,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,question_not_really_a_question,question_opinion_seeking,question_type_choice,question_type_compare,question_type_consequence,question_type_definition,question_type_entity,question_type_instructions,question_type_procedure,question_type_reason_explanation,question_type_spelling,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,0,What am I losing when using extension tubes in...,After playing around with macro photography on...,"I just got extension tubes, so here's the skin...",LIFE_ARTS,1.000000,0.333333,0.000000,0.0,0.0,0.0,1.000000,1.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.0,0.000000,0.0,1.0,0.000000,0.000000,0.0,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.0,0.000000,0.000000,1.000000
1,1,What is the distinction between a city and a s...,I am trying to understand what kinds of places...,It might be helpful to look into the definitio...,CULTURE,1.000000,1.000000,0.000000,0.5,1.0,1.0,0.444444,0.444444,0.666667,0.0,0.000000,0.666667,0.666667,0.0,0.333333,0.0,0.0,0.000000,0.333333,0.0,0.888889,0.888889,0.555556,0.888889,0.888889,0.666667,0.0,0.000000,0.666667,0.888889
2,2,Maximum protusion length for through-hole comp...,I'm working on a PCB that has through-hole com...,Do you even need grooves? We make several pro...,SCIENCE,0.888889,0.666667,0.000000,1.0,1.0,1.0,0.666667,0.444444,0.333333,0.0,0.333333,0.000000,0.000000,0.0,0.000000,0.0,1.0,0.333333,0.333333,0.0,0.777778,0.777778,0.555556,1.000000,1.000000,0.666667,0.0,0.333333,1.000000,0.888889
3,3,Can an affidavit be used in Beit Din?,"An affidavit, from what i understand, is basic...","Sending an ""affidavit"" it is a dispute between...",CULTURE,0.888889,0.666667,0.666667,1.0,1.0,1.0,0.444444,0.444444,0.000000,0.0,0.000000,1.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.888889,0.833333,0.333333,0.833333,1.000000,0.800000,0.0,0.000000,1.000000,1.000000
4,5,How do you make a binary image in Photoshop?,I am trying to make a binary image. I want mor...,Check out Image Trace in Adobe Illustrator. \n...,LIFE_ARTS,1.000000,0.666667,0.000000,1.0,1.0,1.0,0.666667,0.666667,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1.0,0.000000,1.000000,0.0,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.0,0.000000,1.000000,1.000000


In [0]:

drop_test=test.drop(['question_user_name', 'question_user_page','answer_user_name','answer_user_page','url','host','category'], axis=1)
drop_test.head()

,qa_id,question_title,question_body,answer
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,There is no consequence for leaving corpses an...
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,I think it is possible with custom fields.\n\n...
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",You do not have armour in the screenshots. Thi...
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,Your Western Digital hard drive is disappearin...
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,I called two persons who work for Saudia (tick...


In [0]:
import string
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer


def text_cleaner(text):
  rules = [
      {r'>\s+': u'>'},  # remove spaces after a tag opens or closes
      {r'\s+': u' '},  # replace consecutive spaces
      {r'\s*<br\s*/?>\s*': u'\n'},  # newline after a <br>
      {r'</(div)\s*>\s*': u'\n'},  # newline after </p> and </div> and <h1/>...
      {r'</(p|h\d)\s*>\s*': u'\n\n'},  # newline after </p> and </div> and <h1/>...
      {r'<head>.*<\s*(/head|body)[^>]*>': u''},  # remove <head> to </head>
      {r'<a\s+href="([^"]+)"[^>]*>.*</a>': r'\1'},  # show links instead of texts
      {r'[ \t]*<[^<]*?/?>': u''},  # remove remaining tags
      {r'^\s+': u''}  # remove spaces at the beginning
  ]
  for rule in rules:
    for (k, v) in rule.items():
        regex = re.compile(k)
        text = regex.sub(v, text)
    text = text.rstrip()
  return text.lower()

def preprocess_text(column):
  ret_column=[]
  i=0
  max=0
  for text in column:
    i+=1
    # result=text_cleaner(text)
    result = re.sub(r'\d+', '', text)
    result = result.translate(str.maketrans('', '', string.punctuation))
    result.strip()
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(result)
    result = [i for i in tokens if not i in stop_words]
    lemmatizer=WordNetLemmatizer()
    result=lemmatizer.lemmatize(' '.join(result))
    ret_column.append(result)
  return ret_column

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [0]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

drop_data=drop_data_temp.copy()
drop_data['question_title']=preprocess_text(drop_data['question_title'])
drop_data['question_body']=preprocess_text(drop_data['question_body'])
drop_data['answer']=preprocess_text(drop_data['answer'])

drop_test['question_title']=preprocess_text(drop_test['question_title'])
drop_test['question_body']=preprocess_text(drop_test['question_body'])
drop_test['answer']=preprocess_text(drop_test['answer'])

In [0]:

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)
 
def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]
 
    score_vals = []
    feature_vals = []
    
    # word index and corresponding tf-idf score
    for idx, score in sorted_items:
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])
 
    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

# you only needs to do this once, this is a mapping of index to
def get_features(docs,top_words):
  cv=CountVectorizer(max_df=0.85,stop_words=set(stopwords.words("english")),max_features=10000)
  word_count_vector=cv.fit_transform(docs)
  tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
  tfidf_transformer.fit(word_count_vector)
  feature_names=cv.get_feature_names()
  tf_words=[]
  for doc in docs:
    #generate tf-idf for the given document
    tf_idf_vector=tfidf_transformer.transform(cv.transform([doc]))
    #sort the tf-idf vectors by descending order of scores
    sorted_items=sort_coo(tf_idf_vector.tocoo())
    #extract only the top n; n here is 10
    keywords=extract_topn_from_vector(feature_names,sorted_items,top_words)
    tf_idf_values=[]
    words=[]
    tf_idf_values=[0 for x in range(top_words)]
    cnt=0
    for k in keywords:
      words.append(k)
      tf_idf_values[cnt]=keywords[k]
      cnt+=1
    tf_words.append([words,tf_idf_values])
  return tf_words


In [0]:
qt=get_features(drop_data['question_title'],5)
qb=get_features(drop_data['question_body'],20)
a=get_features(drop_data['answer'],20)


qt_te=get_features(drop_test['question_title'],5)
qb_te=get_features(drop_test['question_body'],20)
a_te=get_features(drop_test['answer'],20)

In [0]:
  associated_cols=[[['question_title','question_body'],['question_asker_intent_understanding']],
                  [['question_body'],['question_body_critical']],
                  [['question_title','question_body'],['question_conversational']],
                  [['question_title','question_body'],['question_interestingness_others']],
                  [['question_title','question_body'],['question_interestingness_self']],
                  [['question_title','question_body'],['question_multi_intent']],
                  [['answer','question_title','question_body'],['question_not_really_a_question']],
                  [['answer','question_title','question_body'],['question_opinion_seeking']],
                  [['answer','question_title','question_body'],['question_fact_seeking']],
                  [['question_title','question_body'],['question_well_written']],
                  [['question_title','question_body'],['question_type_entity']],
                  [['question_title','question_body'],['question_type_choice']],
                  [['question_title','question_body'],['question_type_compare']],
                  [['question_title','question_body'],['question_type_consequence']],
                  [['question_title','question_body'],['question_type_definition']],
                  [['question_title','question_body'],['question_type_instructions']],
                  [['question_title','question_body'],['question_type_procedure']],
                  [['answer','question_title','question_body'],['question_type_reason_explanation']],
                  [['question_body'],['question_type_spelling']],
                  # [['answer','both'],['answer_helpful']],
                  # [['answer','both'],['answer_level_of_information']],
                  # [['answer','both'],['answer_type_instructions']],
                  # [['answer','both'],['answer_type_procedure']],
                  # [['answer','both'],['answer_plausible']],
                  # [['answer','both'],['answer_relevance']],
                  # [['answer','both'],['answer_satisfaction']],
                  # [['answer','both'],['answer_type_reason_explanation']],
                  # [['answer','both'],['answer_well_written']],
                  # [['answer','both'],['question_expect_short_answer']],
                  # [['answer','both'],['question_has_commonly_accepted_answer']]],
                  [['question_body','answer','both'],['answer_helpful']],
                  [['question_body','answer','both'],['answer_level_of_information']],
                  [['question_body','answer','both'],['answer_type_instructions']],
                  [['question_body','answer','both'],['answer_type_procedure']],
                  [['question_body','answer','both'],['answer_plausible']],
                  [['question_body','answer','both'],['answer_relevance']],
                  [['question_body','answer','both'],['answer_satisfaction']],
                  [['question_body','answer','both'],['answer_type_reason_explanation']],
                  [['answer'],['answer_well_written']],
                  [['question_body','answer'],['question_expect_short_answer']],
                  [['question_body','answer'],['question_has_commonly_accepted_answer']]]


In [0]:
from scipy import stats
import numpy as np
import math
from scipy.stats import binned_statistic

def min_sparse(X):
    if len(X.data) == 0:
        return 0
    m = X.data.min()
    return m if X.getnnz() == X.size else min(m, 0)

def max_sparse(m):
    i = m.data.argmax()
    return m.data[i]

def pushZerosToEnd(arr): 
  count = 0 # Count of non-zero elements 
  ret_array=[]
  for i in range(len(arr)): 
    if math.isnan(float(arr[i]))== False: 
        # here count is incremented 
        ret_array.append(arr[i]) 
    else:
      count+=1
  for i in range(count):
    ret_array.append(0)

def create_individual_features(arr,df):
  new_feats=[]
  for ind,elem in enumerate(arr):
    print(ind)
    total_words=len(df.iloc[ind].split(' '))
    total_unique_words=len(set(df.iloc[ind].split(' ')))
    if np.count_nonzero(elem[1])!=0:
      average_tf=sum(elem[1]) / np.count_nonzero(elem[1])
      min_=min(elem[1])
      max_=max(elem[1])
      twenty_fifth=stats.scoreatpercentile(np.array(elem[1]), 25)
      fifty=stats.scoreatpercentile(np.array(elem[1]), 50)
      seventy_five=stats.scoreatpercentile(np.array(elem[1]), 75)
      var=np.array(elem[1]).var()
      features=[total_words,total_unique_words,average_tf,min_,max_,twenty_fifth,fifty,seventy_five,var]
      new_feats.append(features)
    else:
      new_feats.append([total_words,total_unique_words,0,0,0,0,0,0,0])
  return new_feats

def get_combined_features(q,a):
  combined_features=[]
  for ind,_ in enumerate(q):
    que=q[ind]
    ans=a[ind]
    common_top_words=list(set(que[0]).intersection(ans[0]))
    num_common=len(common_top_words)
    if num_common!=0:
      indexes_tf=[ans[0].index(x) for x in common_top_words]
      indexes_tf=[ans[1][i] for i in indexes_tf]
      average_tf=sum(indexes_tf) / np.count_nonzero(indexes_tf)
      min_=min(indexes_tf)
      max_=max(indexes_tf)
      combined_features.append([num_common,average_tf,min_,max_])
    else:
      combined_features.append([0,0,0,0])
  return combined_features

In [0]:
from sklearn.preprocessing import MinMaxScaler
qt=get_features(drop_data['question_title'],5)
qb=get_features(drop_data['question_body'],20)
a=get_features(drop_data['answer'],20)

In [0]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_qt=scaler.fit_transform(create_individual_features(qt,drop_data['question_title']))
X_qb=scaler.fit_transform(create_individual_features(qb,drop_data['question_body']))
X_a=scaler.fit_transform(create_individual_features(a,drop_data['answer']))
X_b=scaler.fit_transform(get_combined_features(qb,a))

X_qt_te=scaler.fit_transform(create_individual_features(qt_te,drop_test['question_title']))
X_qb_te=scaler.fit_transform(create_individual_features(qb_te,drop_test['question_body']))
X_a_te=scaler.fit_transform(create_individual_features(a_te,drop_test['answer']))
X_b_te=scaler.fit_transform(get_combined_features(qb_te,a_te))


In [0]:
features_dict_train={'question_title':X_qt,'question_body':X_qb,"answer":X_a,'both':X_b}
features_dict_test={'question_title':X_qt_te,'question_body':X_qb_te,"answer":X_a_te,'both':X_b_te}
submission=pd.DataFrame(columns=['qa_id','question_asker_intent_understanding','question_body_critical','question_conversational','question_expect_short_answer','question_fact_seeking','question_has_commonly_accepted_answer','question_interestingness_others','question_interestingness_self','question_multi_intent','question_not_really_a_question','question_opinion_seeking','question_type_choice','question_type_compare','question_type_consequence','question_type_definition','question_type_entity','question_type_instructions','question_type_procedure','question_type_reason_explanation','question_type_spelling','question_well_written','answer_helpful','answer_level_of_information','answer_plausible','answer_relevance','answer_satisfaction','answer_type_instructions','answer_type_procedure','answer_type_reason_explanation','answer_well_written'])
submission['qa_id']=test['qa_id']

In [0]:
def evaluate(labels,predictions):
    
    labels=np.array(labels)
    predictions=np.array(predictions)
    errors = abs(predictions - labels)
    f = open("results.txt", "a")
    f.write("---------------------------------------------------\n")
    f.write('Model Performance\n')
    f.write('Average Error: {:0.4f} degrees.\n'.format(np.mean(errors)))
    f.write("---------------------------------------------------\n")
    f.close()
    return np.mean(errors)
from sklearn.metrics import fbeta_score, make_scorer
score = make_scorer(evaluate, greater_is_better=False)


In [0]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import fbeta_score, make_scorer
from scipy import stats
import time
import warnings
warnings.filterwarnings('ignore')

def my_model(model,associated_cols,param_grid,features_dict_train,features_dict_test):
  scores=[]
  returned_algorithm={}
  for cases in associated_cols:
    train_cols=[]
    test_cols=[]
    for col in cases[0]:
      train_cols.append(features_dict_train[col])
      test_cols.append(features_dict_test[col])
    X_train= np.concatenate(train_cols, axis=1).tolist()
    X_test= np.concatenate(test_cols, axis=1).tolist()
    Y_train=drop_data[cases[1]]
    from sklearn.model_selection import train_test_split
    xTrain,xTest, yTrain, yTest = train_test_split(X_train, Y_train,stratify=Y_train,test_size=0.80)
    gs = GridSearchCV(model,param_grid,scoring='neg_mean_squared_error',verbose=0)
    start = time.time()
    gs.fit(xTrain, yTrain)
    end = time.time()
    print("======================",cases[1],"=========================")
    print('Time to train model: %0.2fs' % (end -start))
    # for i in ['mean_test_score', 'std_test_score']:
    #     print(i," : ",gs.cv_results_[i].mean(),' with best parameters ',gs.best_params_)
    score=stats.spearmanr(gs.predict(xTest),yTest)
    scores.append(score.correlation)
    print("Evaluation metric using spearman is ",score.correlation,)
    print("Great for ",cases[1])
    myModel=model
    myModel.set_params(**gs.best_params_)
    returned_algorithm[cases[1][0]]=[myModel,score.correlation, type(model).__name__]
    print("======================================================================")
  print("Average score is ",sum(scores) / len(scores) )
  return returned_algorithm

def get_rem_columns(associated_cols):
  for x in associated_cols:
    if x[1][0] not in algorithms.keys():
      new_associated_cols.append(x)
    else:
      print('Column ',x[1][0]," finished.")
  

In [0]:
from sklearn import linear_model
all_algos=[]
param_grid = {#'penalty':['none','l2','l1'],
             'alpha':[0.01,0.1,1],
             'max_iter':[10,100,1000,10000],
             'early_stopping':[True]}
sgd_algorithms=my_model(linear_model.SGDRegressor(),associated_cols,param_grid,features_dict_train,features_dict_test)
all_algos.append(sgd_algorithms)

====================== ['question_asker_intent_understanding'] =========================
Time to train model: 0.83s
Evaluation metric using spearman is  0.10546907653595533
Great for  ['question_asker_intent_understanding']
====================== ['question_body_critical'] =========================
Time to train model: 0.52s
Evaluation metric using spearman is  0.17096877922493925
Great for  ['question_body_critical']
====================== ['question_conversational'] =========================
Time to train model: 0.81s
Evaluation metric using spearman is  0.09161778911706311
Great for  ['question_conversational']
====================== ['question_interestingness_others'] =========================
Time to train model: 1.04s
Evaluation metric using spearman is  0.015754284192028394
Great for  ['question_interestingness_others']
====================== ['question_interestingness_self'] =========================
Time to train model: 0.92s
Evaluation metric using spearman is  0.024727807798

In [0]:
Cs = [0.001, 0.01, 0.1, 1, 10]
gammas = [0.001, 0.01, 0.1, 1]
param_grid = [
  {'C': Cs,'gamma' : gammas, 'kernel': ['linear']},
  {'C':Cs,'gamma' : gammas, 'kernel': ['rbf']}
  ]
svr_algorithms=my_model(SVR(),associated_cols,param_grid,features_dict_train,features_dict_test)
all_algos.append(svr_algorithms)

====================== ['question_asker_intent_understanding'] =========================
Time to train model: 22.20s
Evaluation metric using spearman is  0.17820685056365027
Great for  ['question_asker_intent_understanding']
====================== ['question_body_critical'] =========================
Time to train model: 11.86s
Evaluation metric using spearman is  0.2876567610059798
Great for  ['question_body_critical']
====================== ['question_conversational'] =========================
Time to train model: 7.48s
Evaluation metric using spearman is  0.10825327499066056
Great for  ['question_conversational']
====================== ['question_interestingness_others'] =========================
Time to train model: 14.01s
Evaluation metric using spearman is  0.15461782568116567
Great for  ['question_interestingness_others']
====================== ['question_interestingness_self'] =========================
Time to train model: 14.52s
Evaluation metric using spearman is  0.1607605431

In [0]:
from sklearn.ensemble import RandomForestRegressor
# Number of trees in random forest
n_estimators = [1,10,20,40]
# Maximum number of levels in tree
max_depth = [5,15,25,40]
# Minimum number of samples required to split a node
min_samples_split = [2, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
param_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
rfr_algorithms=my_model(RandomForestRegressor(),associated_cols,param_grid,features_dict_train,features_dict_test)
all_algos.append(rfr_algorithms)

====================== ['question_asker_intent_understanding'] =========================
Time to train model: 110.16s
Evaluation metric using spearman is  0.14823071164352544
Great for  ['question_asker_intent_understanding']
====================== ['question_body_critical'] =========================
Time to train model: 66.25s
Evaluation metric using spearman is  0.31535430555543964
Great for  ['question_body_critical']
====================== ['question_conversational'] =========================
Time to train model: 114.21s
Evaluation metric using spearman is  0.173010261556178
Great for  ['question_conversational']
====================== ['question_interestingness_others'] =========================
Time to train model: 113.23s
Evaluation metric using spearman is  0.14826173757458716
Great for  ['question_interestingness_others']
====================== ['question_interestingness_self'] =========================
Time to train model: 125.79s
Evaluation metric using spearman is  0.174277

In [0]:
import pickle
pkl_filename = "/content/drive/My Drive/pickle_model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(all_algos, file)
for key in svr_algorithms.keys():
  various_per_column=[]
  X_train=[]
  Y_train=[]
  X_Test=[]
  train_cols=[]
  test_cols=[]
  for elem in associated_cols:
    if elem[1][0]==key:
      for col in elem[0]:
        train_cols.append(features_dict_train[col])
        test_cols.append(features_dict_test[col])
      X_train= np.concatenate(train_cols, axis=1).tolist()
      X_test= np.concatenate(test_cols, axis=1).tolist()
      Y_train=drop_data[key]
  for algo in all_algos:
    various_per_column.append(algo[key])
  sorted_algo=sorted(various_per_column, key = lambda x: x[1])
  myModel=sorted_algo[-1][0]
  print('Best algorithm was ',sorted_algo[-1][2],' with training score ',sorted_algo[-1][1],' for ',key)
  myModel.fit(X_train,Y_train)
  predictions=[round(float(pred),5) for pred in myModel.predict(X_test)]
  over_values=False
  final_pred=[]
  for elem in predictions:
    if elem<0 or elem >1:
      if elem <0:
        final_pred.append(0.00001)
      else:
        final_pred.append(0.99999)
      if over_values==True:
        print("Wrong values")
        over_values=True
    else:
      final_pred.append(elem)
  submission[key]=final_pred

Best algorithm was  SVR  with training score  0.17820685056365027  for  question_asker_intent_understanding
Best algorithm was  RandomForestRegressor  with training score  0.31535430555543964  for  question_body_critical
Best algorithm was  RandomForestRegressor  with training score  0.173010261556178  for  question_conversational
Best algorithm was  SVR  with training score  0.15461782568116567  for  question_interestingness_others
Best algorithm was  RandomForestRegressor  with training score  0.17427760463013173  for  question_interestingness_self
Best algorithm was  RandomForestRegressor  with training score  0.17097779252479164  for  question_multi_intent
Best algorithm was  SVR  with training score  0.02123468530612179  for  question_not_really_a_question
Best algorithm was  SVR  with training score  0.189370872956142  for  question_opinion_seeking
Best algorithm was  RandomForestRegressor  with training score  0.13173204525984542  for  question_fact_seeking
Best algorithm was  S

In [0]:
submission=pd.read_csv('submission.csv')
print(submission.shape)
submission.describe()

(476, 31)


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,question_not_really_a_question,question_opinion_seeking,question_type_choice,question_type_compare,question_type_consequence,question_type_definition,question_type_entity,question_type_instructions,question_type_procedure,question_type_reason_explanation,question_type_spelling,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.889925,0.637974,0.222692,0.654009,0.617703,0.592979,0.663784,0.554919,0.344936,0.086207,0.445545,0.311339,0.084410,0.131033,0.048781,0.232930,0.423833,0.176563,0.537802,0.010362,0.848072,0.915616,0.659869,0.955936,0.956194,0.847155,0.392595,0.147765,0.570782,0.889990
std,2812.670060,0.021455,0.071698,0.095515,0.148076,0.105573,0.134567,0.069820,0.036705,0.060510,0.017961,0.150689,0.052995,0.041603,0.077127,0.054963,0.109210,0.081468,0.045131,0.108041,0.016549,0.103269,0.024530,0.075207,0.013787,0.022414,0.053690,0.087168,0.019733,0.130012,0.029270
min,39.000000,0.752310,0.487780,0.021630,0.276420,0.326870,0.333570,0.504870,0.478350,0.107720,0.004430,0.043710,0.185550,0.030460,0.006180,0.012370,0.041560,0.230470,0.083590,0.272160,0.000000,0.511030,0.817890,0.468170,0.875110,0.863210,0.633680,0.262610,0.081420,0.157280,0.789860
25%,2572.000000,0.881305,0.577647,0.162670,0.549772,0.542155,0.485688,0.613645,0.527845,0.334218,0.079710,0.344000,0.276520,0.068040,0.047555,0.013400,0.146530,0.360877,0.156745,0.455815,0.000000,0.777197,0.902100,0.598080,0.951165,0.947730,0.817168,0.327952,0.135300,0.517860,0.871030
50%,5093.000000,0.891835,0.639630,0.227645,0.659920,0.621900,0.572495,0.655935,0.553335,0.359055,0.090135,0.433565,0.303585,0.084575,0.158880,0.019340,0.218820,0.423570,0.175250,0.536465,0.000160,0.856015,0.922410,0.652150,0.960125,0.960735,0.860595,0.366135,0.152525,0.624370,0.891435
75%,7482.000000,0.901182,0.687057,0.290470,0.762750,0.703735,0.695915,0.704730,0.579620,0.380527,0.096685,0.543967,0.335090,0.096552,0.184880,0.079735,0.297937,0.487728,0.184262,0.638653,0.019420,0.928535,0.933593,0.721832,0.965923,0.973137,0.883918,0.434060,0.159832,0.663920,0.910827
max,9640.000000,0.970140,0.844350,0.486950,0.972210,0.851450,0.877540,0.887610,0.671700,0.458100,0.134890,0.937910,0.529870,0.749000,0.466340,0.465220,0.612720,0.659210,0.506060,0.727340,0.092420,0.999990,0.955640,0.840350,0.970340,0.981740,0.999990,0.691010,0.198960,0.731300,0.970680


In [0]:
submission.to_csv ('submission.csv', index = None, header=True,float_format='%.5f')